In [1]:
import pandas as pd
import numpy as np
import parameters as P
import sensitivity_parameters as SP
import helper_functions as hf

In [2]:
io_data = pd.read_csv(P.io_table_dollars_path)

In [3]:
cost = hf.csv_dict_list(P.cost_impact_path)

In [4]:
y = {}
for item in cost.keys():
    y.update({item:0})

In [5]:
m = {} 
for scenario in P.scenario_range:
    new_data = np.zeros([7,3])
    m[scenario] = pd.DataFrame(new_data, columns=P.selectivity, index=P.processes)

In [6]:
for selectivity in P.selectivity:
    for scenario in P.scenario_range:
        if (selectivity == "waterwash"): 
            y["lysine.us.kg"] = SP.other_params[selectivity]['chlys_amount'][scenario] * 0.58 
            # 58% lysine + 42% ChOH for Chylys production (Stoichiometry) 
            y["cholinium.hydroxide.kg"] = (SP.other_params[selectivity]['chlys_amount'][scenario] * 0.42)  
            # 58% lysine + 42% ChOH for Chylys production (Stoichiometry) 
            y["cellulase.kg"] = SP.common_params['enzyme'][scenario]
            y["csl.kg"] = 0.06
            y["farmedstover.kg"] = SP.other_params[selectivity]['feedstock'][scenario]  
            y["dap.kg"] = 0.01
            y["h2so4.kg"] = 0.002
            y["naturalgas.MJ"] = (hf.FuelConvertMJ(
                    SP.other_params[selectivity]['ng_input_stream_mass_ww_kg'][scenario], "naturalgas","kg"))
            y["rail.mt_km"] = ((SP.other_params[selectivity]['chlys_amount'][scenario]/1000) * 100 * 1.6 +
                                  (P.etoh_feed_stream_mass_kg/1000 * SP.common_params['etoh_distribution_rail'][scenario])) 
            y["flatbedtruck.mt_km"] = ((SP.other_params[selectivity]['chlys_amount'][scenario]/1000) * 50 * 1.6 +
                    (P.etoh_feed_stream_mass_kg/1000 * SP.common_params['etoh_distribution_truck'][scenario]))
            y["electricity.{}.kWh".format(P.facility_electricity)] = (
                    SP.other_params[selectivity]['electricity_requirements'][scenario])

            biorefinery_direct_ghg = hf.FuelCO2kg(hf.FuelConvertMJ(
                    SP.other_params[selectivity]['ng_input_stream_mass_ww_kg'][scenario],"naturalgas","kg"), "naturalgas")
            
        
        if (selectivity == "iHG-Projected"):
            y["lysine.us.kg"] = SP.other_params[selectivity]['chlys_amount'][scenario] * 0.58 
            # 55% lysine + 45% ChOH for Chylys production (assumption) 
            y["cholinium.hydroxide.kg"] = SP.other_params[selectivity]['chlys_amount'][scenario] * 0.42
            # 50% lysine + 50% ChOH for Chylys production (assumption) 
            y["cellulase.kg"] = SP.common_params['enzyme'][scenario]
            y["csl.kg"] = 0.05
            y["farmedstover.kg"] = SP.other_params[selectivity]['feedstock'][scenario]
            y["dap.kg"] = 0.017  
            y["rail.mt_km"] = ((SP.other_params[selectivity]['chlys_amount'][scenario]/1000) * 100 * 1.6 +
                               (P.etoh_feed_stream_mass_kg/1000 * SP.common_params['etoh_distribution_rail'][scenario]))
            y["flatbedtruck.mt_km"] = ((SP.other_params[selectivity]['chlys_amount'][scenario]/1000) * 50 * 1.6 +
                    (P.etoh_feed_stream_mass_kg/1000 * SP.common_params['etoh_distribution_truck'][scenario]))
            y["electricity.{}.kWh".format(P.facility_electricity)] = (
                    SP.other_params[selectivity]['electricity_requirements'][scenario])

            biorefinery_direct_ghg = 0
                    
        if (selectivity == "iHG-Current"):
            # Data needs to be updated for H2SO4! 
            y["lysine.us.kg"] = SP.other_params[selectivity]['chlys_amount'][scenario] * 0.58        
            # 55% lysine + 45% ChOH for Chylys production (assumption) 
            y["cholinium.hydroxide.kg"] = (SP.other_params[selectivity]['chlys_amount'][scenario] * 0.42)  
            # 50% lysine + 50% ChOH for Chylys production (assumption) 
            y["cellulase.kg"] = SP.common_params['enzyme'][scenario]
            y["csl.kg"] = 0.05
            y["farmedstover.kg"] = SP.other_params[selectivity]['feedstock'][scenario]
            y["dap.kg"] = 0.01
            y["hcl.kg"] = 0.18
            y["rail.mt_km"] = ((SP.other_params[selectivity]['chlys_amount'][scenario]/1000) * 100 * 1.6 + (
                P.etoh_feed_stream_mass_kg/1000 * SP.common_params['etoh_distribution_rail'][scenario]))
            y["flatbedtruck.mt_km"] = ((SP.other_params[selectivity]['chlys_amount'][scenario]/1000) * 50 * 1.6 +
                    (P.etoh_feed_stream_mass_kg/1000 * SP.common_params['etoh_distribution_truck'][scenario])) 

            y["electricity.{}.kWh".format(P.facility_electricity)] = (
                    SP.other_params[selectivity]['electricity_requirements'][scenario])

        results_kg_co2e = hf.TotalGHGEmissions(io_data, y, cost, 
                                               biorefinery_direct_ghg, P.combustion_direct_ghg)
        
        results_kg_co2e_dict = results_kg_co2e.set_index('products')['ghg_results_kg'].to_dict()

        m = hf.AggregateResults(m, results_kg_co2e_dict, selectivity, scenario)
        
        m[scenario][selectivity] = m[scenario][selectivity] * 1000/27 # converting kg per kg results to g per MJ

[  1.35763743e-04   0.00000000e+00   4.52545810e-05   2.71527486e-06
   0.00000000e+00   2.70000000e-02   0.00000000e+00   0.00000000e+00
   0.00000000e+00   4.46042080e-04   0.00000000e+00   1.34531615e-04
   0.00000000e+00   4.42138128e-06   5.19035000e-02   1.70000000e-02
   1.40547803e-02   1.13815850e-02   0.00000000e+00   8.14230277e-01
   0.00000000e+00   2.89431623e-03   0.00000000e+00   6.03771276e-01
   2.59082955e-05   0.00000000e+00   0.00000000e+00   1.97447758e-01
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   5.40000000e-04   2.16000000e-04   0.00000000e+00   0.00000000e+00
   0.00000000e+00   2.25062017e-03   4.79093661e-01   6.63562526e-03
   8.71576581e-03   1.68792380e-03   0.00000000e+00   0.00000000e+00
   3.80435390e-01   0.00000000e+00   0.00000000e+00   2.26139002e-02
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   7.55463351e-07   2.82768720e-03   0.00000000e+00
   5.37291388e-05   0.00000000e+00

In [ ]:
cost

In [ ]:
y_array = []
cost_array = []
for item in io_data['products']:
    y_array.append(y[item])
    cost_array.append(cost[item])

In [ ]:
result = []

# get last index for the lists for iteration
end_index = len(y_array)

for i in range(end_index):
    result.append(y_array[i]/cost_array[i])
result

In [ ]:
m['avg']

In [ ]:
n = {} 
for scenario in P.scenario_range:
    n_new_data = np.zeros([3,3])
    n = pd.DataFrame(n_new_data, columns=P.selectivity, index=P.scenario_range)
        

In [ ]:
biorefinery_direct_ghg = 0
for selectivity in P.selectivity:
    for scenario in P.scenario_range:
        y["electricity.{}.kWh".format(P.facility_electricity)] = (
                -cost["electricity.{}.kWh".format(P.facility_electricity)] * (
                    SP.other_params[selectivity]['electricity_credit'][scenario]))
        
        results_kg_co2e = hf.TotalGHGEmissions(io_data, y, cost, 
                                               biorefinery_direct_ghg, P.combustion_direct_ghg)
        results_kg_co2e_dict = results_kg_co2e.set_index('products')['ghg_results_kg'].to_dict()
        n[selectivity][scenario] = results_kg_co2e_dict["electricity.US.kWh"]

In [ ]:
results_kg_co2e_dict

In [ ]:
results_kg_co2e_dict["electricity.US.kWh"].sum()

In [ ]:
A = np.matrix([[0,2,2],[2,0,2],[2,2,0]])

In [ ]:
A

In [ ]:
I = np.zeros((A.shape[1], A.shape[1]), int)
np.fill_diagonal(I, 1)

In [ ]:
I

In [ ]:
(I-A)

In [ ]:
y=[1,1,1]

In [ ]:
np.linalg.solve((I - A), y)